<a href="https://colab.research.google.com/github/Nikhitha-S-Pavan/Deep-learning-examples-using-keras/blob/main/natural_images_classification_using_vgg16_acc%3E95_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# import the libraries as shown below

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob

In [ ]:
# Import the Vgg 16 library as shown below and add preprocessing layer to the front of VGG
# Here we will be using imagenet weights
IMAGE_SIZE=[128,128]
vgg16 = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [ ]:
# don't train existing weights
for layer in vgg16.layers:
    layer.trainable = False

In [ ]:
# useful for getting number of output classes
folders = glob("/content/drive/My Drive/kaggle_dataset/natural_images/data/natural_images/*")
len(folders)

In [ ]:
# our layers - you can add more if you want
x = Flatten()(vgg16.output)

prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=vgg16.input, outputs=prediction)
# view the structure of the model
model.summary()

In [ ]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [ ]:
# Preprocessing the Training set
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   zoom_range = 0.2,
                                   validation_split=0.2,
                                   horizontal_flip = True)

training_set = train_datagen.flow_from_directory('/content/drive/My Drive/kaggle_dataset/natural_images/data/natural_images/',
                                                 target_size = (128,128),
                                                 batch_size =64,
                                                 
                                                 subset="training")


test_set = train_datagen.flow_from_directory('/content/drive/My Drive/kaggle_dataset/natural_images/data/natural_images/',
                                            target_size = (128,128),
                                            batch_size = 64,
                                            
                                            subset="validation")

In [ ]:
# fit the model
# Run the cell. It will take some time to execute
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=5,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

In [ ]:
model.save("/content/drive/My Drive/kaggle_dataset/natural_images/data/natural_images.h5")

In [ ]:
# predict image class
from numpy import loadtxt
from keras.models import load_model
 
# load model
model = load_model('/content/drive/My Drive/kaggle_dataset/natural_images/data/natural_images.h5')

In [ ]:
# summarize model.
model.summary()

In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
# load and prepare the image
def load_image(filename):
  
	# load the image
	img = load_img(filename, target_size=(128,128))
	# convert to array
	img = img_to_array(img)
	# reshape into a single sample with 3 channels
	img = img.reshape(1, 128,128, 3)
	# center pixel data
	img = img.astype('float32')
	img = img - [123.68, 116.779, 103.939]
	return img
 


In [ ]:
# load an image and predict the class
def run_example():
	# load the image
	img = load_image('/content/drive/My Drive/kaggle_dataset/natural_images/resized/airplane/airplane_0000.jpg')
	# load model
	model = load_model('/content/drive/My Drive/kaggle_dataset/natural_images/data/natural_images_cnn.h5')
	# predict the class
	result = model.predict(img)
	print(result[0])
  
 
# entry point, run the example
run_example()